In [1]:
# hard negative 만들기
from utils.utils import *
from tqdm import tqdm
from transformers import AutoTokenizer
import argparse
from retrieval.sparse_retrieval.model import BM25
from sklearn.model_selection import train_test_split
import json

In [2]:
parser = argparse.ArgumentParser()
# test name
parser.add_argument('--contexts_path', type=str, default='../data/KorQuAD_dataset/data_w_context/korquad_contexts.jsonl')
parser.add_argument('--test_data_path', type=str, default='../data/KorQuAD_dataset/data_w_context/dev.jsonl')

_StoreAction(option_strings=['--test_data_path'], dest='test_data_path', nargs=None, const=None, default='../data/KorQuAD_dataset/data_w_context/dev.jsonl', type=<class 'str'>, choices=None, help=None, metavar=None)

In [3]:
args,_ = parser.parse_known_args()

In [4]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [5]:
contexts = load_jsonl(args.contexts_path)

10581it [00:00, 143822.18it/s]


In [6]:
bm25 = BM25(tokenizer)
bm25.exec_embedding(contexts)

Token indices sequence length is longer than the specified maximum sequence length for this model (752 > 512). Running this sequence through the model will result in indexing errors


In [8]:
test_data = load_jsonl(args.test_data_path)

5774it [00:00, 108104.09it/s]


# test_data score 기록

In [9]:
result = []
for i in tqdm(test_data):
    output = bm25.retrieve(i['question'],100)
    output = [i[0] for i in output]
    result.append(output)

100%|██████████████████████████████████████████████████████████████████████████████| 5774/5774 [05:41<00:00, 16.89it/s]


In [11]:
# hit@1, 5, 20, 100
scores = []
for k in range(1, 101):
    score_k = []
    for i,j in tqdm(zip(test_data, result)):
        score_k.append(i['positive_ctxs_ids'][0] in j[:k])
    scores.append(score_k)

5774it [00:00, 736039.61it/s]
5774it [00:00, 1011059.63it/s]
5774it [00:00, 1335055.75it/s]
5774it [00:00, 637565.13it/s]
5774it [00:00, 1155159.14it/s]
5774it [00:00, 1150439.95it/s]
5774it [00:00, 1151972.19it/s]
5774it [00:00, ?it/s]
5774it [00:00, 443161.90it/s]
5774it [00:00, 1285724.74it/s]
5774it [00:00, 960990.09it/s]
5774it [00:00, 1352804.79it/s]
5774it [00:00, 620383.52it/s]
5774it [00:00, 962173.67it/s]
5774it [00:00, 1023147.92it/s]
5774it [00:00, 1114851.14it/s]
5774it [00:00, ?it/s]
5774it [00:00, 1037658.48it/s]
5774it [00:00, ?it/s]
5774it [00:00, 443738.41it/s]
5774it [00:00, 1873146.52it/s]
5774it [00:00, ?it/s]
5774it [00:00, 822563.39it/s]
5774it [00:00, 1583387.47it/s]
5774it [00:00, 636576.37it/s]
5774it [00:00, ?it/s]
5774it [00:00, 424198.41it/s]
5774it [00:00, 2480326.84it/s]
5774it [00:00, ?it/s]
5774it [00:00, 378056.34it/s]
5774it [00:00, ?it/s]
5774it [00:00, 495618.68it/s]
5774it [00:00, 1155489.83it/s]
5774it [00:00, 1440513.40it/s]
5774it [00:00, 836427

In [12]:
for i in scores:
    print(sum(i)/len(i))

0.85417388292345
0.9243158988569449
0.9461378593695878
0.9547973675095255
0.9615517838586768
0.9658815379286456
0.9695185313474195
0.9722895739521995
0.9745410460685833
0.9760997575337721
0.9771388985105646
0.9783512296501559
0.9790439903013509
0.9799099411153447
0.9802563214409421
0.9809490820921372
0.982507793557326
0.9828541738829234
0.9832005542085209
0.9842396951853135
0.9844128853481122
0.9847592656737097
0.9849324558365085
0.9849324558365085
0.9856252164877035
0.9861447869760998
0.9863179771388985
0.986664357464496
0.9870107377900935
0.987357118115691
0.9875303082784898
0.9877034984412886
0.9882230689296848
0.9885694492552823
0.9885694492552823
0.988742639418081
0.988742639418081
0.9889158295808798
0.9892622099064773
0.9892622099064773
0.9894354000692761
0.9894354000692761
0.9896085902320748
0.9896085902320748
0.9897817803948735
0.9899549705576723
0.9903013508832699
0.9903013508832699
0.9903013508832699
0.9904745410460686
0.9908209213716661
0.9909941115344648
0.9909941115344648


# hard negative attach

In [6]:
train_data = load_jsonl('./data/train.jsonl')

51237it [00:00, 152080.37it/s]


In [7]:
for i in tqdm(train_data):
    pos,neg = bm25.retrieve_without(i['question'], i['answer'], 100, 1)
    i['hard_negative_ctxs_ids'] = neg
    i['retrieved_ctxs_ids'] = pos

100%|█████████████████████████████████| 51237/51237 [141:23:15<00:00,  9.93s/it]


In [28]:
train_data, val_data = train_test_split(train_data, test_size = 0.1, random_state = 42, shuffle = True)

In [21]:
wiki = json.load(open(args.data_path))

In [29]:
for i in tqdm(train_data):
    i['negative_ctxs_ids'] = i['hard_negative_ctxs_ids']
    del i['hard_negative_ctxs_ids']
    i['negative_ctxs'] = [wiki[j] for j in i['negative_ctxs_ids']]
    i['sparse_retrieved_ctxs_ids'] = i['retrieved_ctxs_ids']
    del i['retrieved_ctxs_ids']

100%|█████████████████████████████████| 41501/41501 [00:00<00:00, 305810.26it/s]


In [30]:
for i in tqdm(val_data):
    i['negative_ctxs_ids'] = i['hard_negative_ctxs_ids']
    del i['hard_negative_ctxs_ids']
    i['negative_ctxs'] = [wiki[j] for j in i['negative_ctxs_ids']]
    i['sparse_retrieved_ctxs_ids'] = i['retrieved_ctxs_ids']
    del i['retrieved_ctxs_ids']

100%|███████████████████████████████████| 4612/4612 [00:00<00:00, 194561.98it/s]


In [5]:
train_data = load_jsonl('./data/train_data_attached.jsonl')
val_data = load_jsonl('./data/val_data_attached.jsonl')

41501it [00:05, 7785.65it/s] 
4612it [00:00, 5766.54it/s] 


In [9]:
val_data = load_jsonl('./data/dev.jsonl')

4909it [00:00, 71522.05it/s]


In [ ]:
for i in train_data:
    i['neagtive

In [34]:
save_jsonl('./data',train_data, 'train_data_attached')
save_jsonl('./data',val_data, 'val_data_attached')

# check_hard_negative

In [22]:
import json

In [24]:
wiki = json.load(open(args.data_path))

In [25]:
wiki['오우데나르데 전투_19']

{'title': '오우데나르데 전투',
 'context': '군대는 오우데나르데 근처의 임시 다리가 붕괴되는 바람에 강을 건널 수가 없었고, 도시의 돌다리를 이용하여 강을 건너야 했기 때문에 한시간정도 늦어지게 되었다. 말버러 공작은 계획에 착수하여 외젠 공작의 기병대에게 돌격을 명했다. 이 돌격은 부르고뉴 공작의 사령부에 행해졌다. 프랑스군의 본진을 지키던 기병대는 메종 드 로이(Maison du Roi)부대였는데, 이들은 외젠 공작의 기병대에 대하여'}

In [30]:
check = []
for i in tqdm(train_data):
    assert i['answer'] not in wiki[i['hard_negative_ctxs_ids'][0]]['context']

100%|█████████████████████████████████| 50724/50724 [00:00<00:00, 339804.40it/s]
